In [1]:
import cv2
from skimage.transform import resize
from os import listdir, walk, makedirs, replace
from os.path import isfile, join
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import random
import pickle
import itertools
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
import tensorflow as tf

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img



In [8]:
# TODO make a list of the mushrooms names
directory = 'clean_data/'
folder_list= [f for f in listdir(directory)]
# folder_list.remove('mushroom_sample')
# folder_list.remove('mushroom_sample.zip')
folder_list.remove('.DS_Store')
folder_list.remove('data_pool')
folder_list
CLASSES = dict(zip(folder_list, range(len(folder_list)))) # dictionary of the mushrooms with labels

In [9]:
# TODO: call its folder category separately and make an array of items from the images
def get_label(key ,class_mush = CLASSES):
    """
    You can get the labeled name of the mushroom by giving 
    the index or the index by giving the key
    """
    if type(key) == str:
        return class_mush[key]
    elif type(key) == int:
        return list(CLASSES.keys())[list(CLASSES.values()).index(key)]


def get_images(fl ):
    """
    give the list of photos from directory and the labels.
    fl = folder of images
    """
    directory = 'data/Data/Mushrooms/'
    label = []
    mush= []
    pictures_dir = directory + fl + '/'
    pictures = [pics for pics in listdir(pictures_dir) if isfile(join(pictures_dir , pics))]
    for i in pictures:
        mush.append(cv2.imread(pictures_dir+i))
        label.append(get_label(fl))     
    return mush, label, len(label)



In [ ]:
# TODO split data train valid test (75% train, 15% valid, 10% test)
def split_preprocessed_data(X  ,y , test_size = 0.1):
    """
    This function returns the train and validataion data 
    preprocessed if necessary: in a flattened 1_D form
    with normalized X and y per 255.
    Split images (75%/15%/10%) and save to temporary folders
    """
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, random_state=42)
    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = test_size * 1.7, random_state=42)  
    where ='clean_data/data_pool/'+ get_label(y[0]) + '/'
    with open('X_train','wb') as f: pickle.dump(X_train, f)
    os.makedirs(where + 'train/', exist_ok=True)
    os.replace('X_train', where+ 'train/X_train')
    with open('y_train','wb') as f: pickle.dump(y_train, f)
    os.replace('y_train', where+ 'train/y_train')
    os.makedirs(where + 'test/', exist_ok=True)
    with open('X_test','wb') as f: pickle.dump(X_test, f)
    os.replace('X_test', where+ 'test/X_test')
    with open('y_test','wb') as f: pickle.dump(y_test, f)
    os.replace('y_test', where+ 'test/y_test')
    os.makedirs(where + 'valid/', exist_ok=True)
    with open('X_valid','wb') as f: pickle.dump(X_valid, f)
    os.replace('X_valid', where+ 'valid/X_valid')
    with open('y_valid','wb') as f: pickle.dump(y_valid, f)
    os.replace('y_valid', where+ 'valid/y_valid')   
    return X_train, X_test, y_train, y_test, X_valid, y_valid

In [ ]:
def augmented_images(image, number_of_extra, save_to_dir, save_prefix):

    datagen = ImageDataGenerator(
        rotation_range=40,
        horizontal_flip=True,
        vertical_flip=True)
    image = image
    l = image.reshape((1,) + image.shape)  # this is a Numpy array with shape (1, 3, 150, 150)


    # the .flow() command below generates batches of randomly transformed images
    # and saves the results to the `preview/` directory
    i = 0

    for batch in datagen.flow(l, batch_size=1,
            save_to_dir=save_to_dir, save_prefix=save_prefix, save_format='jpg'):     

            i += 1
            if i > number_of_extra-1:
                    break  # otherwise the generator would loop indefinitel
    

In [ ]:
# TODO: add augmented images in the X_train y_train arrays
def get_augmented_images(fl, directory='clean_data/data_pool/'):
    """
    give the list of photos from directory and the labels.
    fl = folder of images with naming the mashroom
    """
    directory = directory #'data/Data/Mushrooms/'

    label = []
    mush= []
    pictures_dir = directory + fl + '/test/'
    pictures = [pics for pics in listdir(pictures_dir)]# if isfile(join(pictures_dir , pics))]
    for i in pictures:
        mush.append(cv2.imread(pictures_dir+i))
        label.append(get_label(fl))     
    return mush, label, len(label)



In [ ]:
def concat_save(X_train=X_train,mush=mush,y_train=y_train,label=label):
    X_train = np.concatenate((X_train,mush))
    y_train = np.concatenate((y_train,label))

    where ='clean_data/data_pool/'+ get_label(label[0]) + '/test_full/'
    os.makedirs(where , exist_ok=True)
    with open('X_train','wb') as f: pickle.dump(X_train, f)
    os.replace('X_train', where + 'X_train')
    with open('y_train','wb') as f: pickle.dump(y_train, f)
    os.replace('y_train', where + 'y_train')
    return X_train, y_train
    

In [ ]:
# Do the process for all mushrooms
for f in folder_list:
    mush, label, length = get_images(f)
    X_train, X_test, y_train, y_test, X_valid, y_valid = split_preprocessed_data(mush,label)
    # TODO: augmentation for the mushrooms with small number of images in train and valid
    if len(y_train)/750 < 0.8:
        for i in range(len(y_train)):
            augmented_images(X_train[i] ,2 , 'clean_data/data_pool/'+f+'/train','train')

    mush, label ,length =get_augmented_images(f)
    X_train, y_train = concat_save()

# took 22 minutes to run the code

In [ ]:
# image reshape per folder

def image_reshape_train(image, new_size = (224,224)):
    """
    give the list of photos with the correct pixel size you want to 
    downsize. 
    image is the folder with the name of mushroom as string, i.e. 'Agaricus'
    """
    directory = 'clean_data/data_pool/'+ image + '/'
    if os.path.exists(directory + 'train_full/'):
        directory = directory + 'train_full/'
    elif os.path.exists(directory + 'train/'):
        directory = directory + 'train/'

    with open(directory + 'X_train', "rb") as fp:  X_train = pickle.load(fp)
    with open(directory + 'y_train', "rb") as fp:  y_train = pickle.load(fp)
    image = X_train
    label = y_train
    items_resized = []
    for idx, dir in enumerate(image):     
        item_resized_colored = resize(dir, new_size)
        items_resized.append(item_resized_colored)
    directory = 'clean_data/data_pool/'+ get_label(label[0]) + '/'
    with open(directory + "X_train_resized", "wb") as fp:   #Pickling
        pickle.dump(items_resized, fp)
    
# for folder in folder_list:
#     image_reshape_train(folder, )
    
def image_reshape(image, new_size = (224,224)):
    """
    give the list of photos with the correct pixel size you want to 
    downsize. 
    image is the folder with the name of mushroom as string, i.e. 'Agaricus'
    """
    # for the test
    # directory = 'clean_data/data_pool/'+ image + '/test/'
    # with open(directory + 'X_test', "rb") as fp:  X_test = pickle.load(fp)
    # with open(directory + 'y_test', "rb") as fp:  y_test = pickle.load(fp)
    # image = X_test
    # label = y_test
    # items_resized = []
    # for idx, dir in enumerate(image):     
    #     item_resized_colored = resize(dir, new_size)
    #     items_resized.append(item_resized_colored)
    # directory = 'clean_data/data_pool/'+ get_label(label[0]) + '/'
    # with open(directory + "X_test_resized", "wb") as fp:
    #     pickle.dump(items_resized, fp)
    #     # for the valid
    directory = 'clean_data/data_pool/'+ image + '/valid/'
    with open(directory + 'X_valid', "rb") as fp:  X_valid = pickle.load(fp)
    with open(directory + 'y_valid', "rb") as fp:  y_valid = pickle.load(fp)
    image = X_valid
    label = y_valid
    items_resized = []
    for idx, dir in enumerate(image):     
        item_resized_colored = resize(dir, new_size)
        items_resized.append(item_resized_colored)
    directory = 'clean_data/data_pool/'+ get_label(label[0]) + '/'
    with open(directory + "X_valid_resized", "wb") as fp:
        pickle.dump(items_resized, fp)
    


In [ ]:
for f in folder_list:
    image_reshape_train(f)

# 10 minutes to run the code

In [ ]:
for f in folder_list:

    image_reshape(f)

In [4]:
# TODO: unpickle and connect
def unpickle(directory, category ):
    """
    This functions takes the data from a directory that have been
    downsized in banches ang gives back the X,y as arrays.
    """
    table_mushrooms_downsized = []
    label_mushrooms = []
    for mushroom in CLASSES:

        dir = directory + mushroom + '/X_' + category + '_resized'
        with open(dir, "rb") as fp:  X_resized = pickle.load(fp)
        table_mushrooms_downsized.append(X_resized)
        label = np.full((1, len(X_resized)), get_label(mushroom))#.ravel()
        label_mushrooms = np.append(label_mushrooms,label)

    # flatten the nested list into a 1D list
    mushrooms_downsized = list(itertools.chain(*table_mushrooms_downsized))
    return mushrooms_downsized, label_mushrooms

    # if len(mushrooms_downsized) == len(label):

    # #     #Check if the number of pictures and labels are the same.
    #     X = np.asarray(mushrooms_downsized)
    #     y = np.asarray(label_mushrooms)

    #return X, y



In [ ]:
for ca in ["train","valid"]:
    category = ca
    X, y = unpickle(directory="clean_data/data_pool/", category = category)
    directory = 'clean_data/data_pool/'
    # shuffle the samples and labels together. Since it 
    # shuffler = np.random.permutation(len(X))
    # X = np.array(X)
    # y = np.array(y)
    # X = X[shuffler]
    # y = y[shuffler]
    with open(directory + "X" + category, "wb") as fp: pickle.dump(X, fp)
    with open(directory + "y" + category, "wb") as fp: pickle.dump(y, fp)

In [ ]:
# shuffle the samples and labels together
shuffler = np.random.permutation(len(X))
X = np.array(X)
y = np.array(y)
X = X[shuffler]
y = y[shuffler]
with open(directory + "X" + category, "wb") as fp: pickle.dump(X, fp)
with open(directory + "y" + category, "wb") as fp: pickle.dump(y, fp)

In [ ]:
plt.imshow(X[1000],cmap='Greys')


In [5]:
dir ='clean_data/data_pool/Xtrain'
with open(dir, "rb") as fp:  X_train = pickle.load(fp)

In [15]:
len(X_train)

NameError: name 'X_train' is not defined

In [13]:
# TODO: model


In [11]:
# TODO: unpickle and connect
def unpickle(directory, category ):
    """
    This functions takes the data from a directory that have been
    downsized in banches ang gives back the X,y as arrays.
    """
    label_mushrooms = []
    for mushroom in CLASSES:

        dir = directory + mushroom + '/X_' + category + '_resized'
        with open(dir, "rb") as fp:  X_resized = pickle.load(fp)
        label = np.full((1, len(X_resized)), get_label(mushroom))#.ravel()
        label_mushrooms = np.append(label_mushrooms,label)

    return  label_mushrooms

    # if len(mushrooms_downsized) == len(label):

    # #     #Check if the number of pictures and labels are the same.
    #     X = np.asarray(mushrooms_downsized)
    #     y = np.asarray(label_mushrooms)

    #return X, y
for ca in ["train"]:
    category = ca
    y = unpickle(directory="clean_data/data_pool/", category = category)
    directory = 'clean_data/data_pool/'

    with open(directory + "y" + category, "wb") as fp: pickle.dump(y, fp)



In [12]:
with open('clean_data/data_pool/Xtest', "rb") as fp:  X = pickle.load(fp)
with open('clean_data/data_pool/ytest', "rb") as fp:  y = pickle.load(fp)

EOFError: Ran out of input

In [ ]:
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, decode_predictions, preprocess_input
from tensorflow.keras.preprocessing import image # Keras own inbuild image class
import os
from tensorflow.keras.models import Model
from tensorflow import keras

def model_ResNet50V2(CLASSES = CLASSES,learning_rate=0.001):
    """
    This function returns a ResNet50V2 model with the last
    layer removed.
    """
    K.clear_session() # Always clear the session!

    base_model = ResNet50V2(
        weights='imagenet', 
        pooling='avg',      # applies global average pooling to the output of the last conv layer (like a flattening)
        include_top=False,   # !!!!! we only want to have the base, not the final dense layers 
        input_shape=(224, 224, 3)  
    )
    base_model.trainable = False # To freeze the model
    # Start building on top of the model
    model = keras.Sequential() # defining a new model
    model.add(base_model) # adding in the pretrained model without the fully connected layer
    model.add(keras.layers.Dense(64, activation='relu')) # adding in additional layers
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(128, activation='relu')) # adding in additional layers
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(len(CLASSES), activation='softmax')) #!!! Final layer with a length of 2, and softmax activation 
    # have a look at the trainable and non-trainable params statistic
    model.summary()
    model.compile(optimizer=keras.optimizers.Adam(learning_rate),
              loss=keras.losses.categorical_crossentropy,
              metrics=[keras.metrics.categorical_accuracy])

    return model

def fit_ResNet50V2(X_train, y_train,epochs = 200, batch_size =500,sav = True):
    """
    This function fits the model on the training data.
    """
    # observe the validation loss and stop when it does not improve after 3 iterations
    callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

    history = model.fit(X_train, y_train, 
              epochs=epochs, 
              verbose=2,
              batch_size=batch_size, 
              callbacks=[callback],
              # use 30% of the data for validation
              validation_split=0.3)
    # save model
    if sav == True:
        model.save('models/ResNet50V2_RGB_300fot_SD.h5')
        
    return history
model = model_ResNet50V2()

history = fit_ResNet50V2(X, y, epochs = 200, batch_size =300)